# 0. Install and Import Dependencies

In [17]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
!pip install nbimporter
!pip install nbformat
!pip install pygltflib
!pip install trimesh
!pip install pyrender

In [19]:
import cv2
import mediapipe as mp
import numpy as np
import time
import cv2
import numpy as np
import math
import os
from collections import deque
import logging
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [20]:
import math
import trimesh
import pyrender
from pygltflib import GLTF2


import numpy as np
import trimesh
import pyrender
import time
import cv2  # Import OpenCV for displaying the rendered output
from pygltflib import GLTF2

# Load the glTF file
gltf = GLTF2().load("3D/magicmirror.glb")

# Load the GLTF as a Trimesh scene
scene_trimesh = trimesh.load("3D/magicmirror.glb")
if isinstance(scene_trimesh, trimesh.Scene):
    # If the GLTF file is a scene, combine all geometries into one
    combined_mesh = trimesh.util.concatenate(scene_trimesh.dump())
else:
    # If it's already a Trimesh, use it directly
    combined_mesh = scene_trimesh

# Scale the mesh down
scaling_factor = 0.1  # Adjust this value to control the size
combined_mesh.apply_scale(scaling_factor)

# Create a Pyrender mesh
pyrender_mesh = pyrender.Mesh.from_trimesh(combined_mesh)

# Create a Pyrender scene
scene = pyrender.Scene()
node = scene.add(pyrender_mesh)

# Add a camera to the scene
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
camera_pose = np.eye(4)
camera_pose[:3, 3] = [0, 0, 2]  # Position the camera slightly away from the model
scene.add(camera, pose=camera_pose)

# Add lighting
light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
scene.add(light, pose=camera_pose)

# Create an offscreen renderer
renderer = pyrender.OffscreenRenderer(viewport_width=1000, viewport_height=1000)

# Animate the model by moving it to the left
translation = np.eye(4)  # Transformation matrix for translation
translation[0, 3] = 0  # Start with no translation


# 3D mapping, moves left

# try:
#     while True:
#         # Move the model slowly to the left
#         translation[0, 3] -= 0.01  # Decrease x-coordinate
#         scene.set_pose(node, pose=translation)

#         # Render the scene
#         color, _ = renderer.render(scene)

#         # Display with OpenCV
#         color_bgr = color[..., ::-1]  # Convert RGB to BGR
#         cv2.imshow("GLTF Viewer", color_bgr)

#         # Quit on pressing 'q'
#         if cv2.waitKey(10) & 0xFF == ord("q"):
#             break

#         time.sleep(0.01)  # Slow down animation
# except KeyboardInterrupt:
#     pass
# finally:
#     renderer.delete()
#     cv2.destroyAllWindows()







# gltf = GLTF2().load("3D\magicmirror.glb")

# Getting skeleton
m_index = 0
material = gltf.materials[m_index]
material.alphaMode = "MASK"
material.alphaCutoff=0.5
for node_idx, node in enumerate(gltf.nodes):
    if node.skin is not None:
        joints = gltf.skins[node.skin].joints
        bone_list=[]
        for joint in joints:
            joint_node = gltf.nodes[joint]
            bone_list.append(joint_node)
    if node.children:
        node.children = [child for child in node.children if child != 9 ]


# for bone in bone_list:
#     print(f"bone {bone.name} rotation:{bone.rotation} and translation:{bone.translation}")

if 9 not in gltf.scenes[0].nodes:
    gltf.scenes[0].nodes.append(9)
    print("Not in list")
else:
    print("In list")

# Access the first mesh in the scene (or iterate if you have more meshes)
mesh = gltf.meshes[0]
for primitive in mesh.primitives:
    position = gltf.accessors[primitive.attributes.POSITION]
    print(f"Position {position}")
gltf.save("3D\magicmirror.glb")

def active_update():
    shirt_mesh = trimesh.load('3D\magicmirror.glb')
    scene = pyrender.Scene()
    scene.add(pyrender.Mesh.from_trimesh(trimesh.util.concatenate(shirt_mesh.dump())))
    camera = pyrender.PerspectiveCamera(yfov=60, aspectRatio=1.0)
    pose = np.eye(4)
    pose[:3,3]=[0,0,2]
    scene.add(camera, pose=pose)

    renderer = pyrender.OffscreenRenderer(viewport_width=640,viewport_height=480)
    color,_=renderer.render(scene)

    color_bgr=cv2.cvtColor(color, cv2.COLOR_RGB2BGR)

    cv2.imshow("gltf",color_bgr)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cv2.destroyAllWindows()



In list

<>:119: SyntaxWarning: invalid escape sequence '\m'
<>:122: SyntaxWarning: invalid escape sequence '\m'
<>:119: SyntaxWarning: invalid escape sequence '\m'
<>:122: SyntaxWarning: invalid escape sequence '\m'
C:\Users\mertz\AppData\Local\Temp\ipykernel_47884\2480808554.py:119: SyntaxWarning: invalid escape sequence '\m'
  gltf.save("3D\magicmirror.glb")
C:\Users\mertz\AppData\Local\Temp\ipykernel_47884\2480808554.py:122: SyntaxWarning: invalid escape sequence '\m'
  shirt_mesh = trimesh.load('3D\magicmirror.glb')



Position Accessor(extensions={}, extras={}, bufferView=0, byteOffset=0, componentType=5126, normalized=False, count=9192, type='VEC3', sparse=None, max=[1.0807095766067505, 2.9985105991363525, 1.5579530000686646], min=[-0.8559107184410095, -0.38889896869659424, -1.6121143102645874], name=None)


# 1. Swipe Detection

In [21]:
def leftSwipeDetection(leftWristLocations):
    return leftWristLocations[0][0] - leftWristLocations[-1][0] > 100

def rightSwipeDetection(rightWristLocations):
    return rightWristLocations[-1][0] - rightWristLocations[0][0] > 100

def upSwipeDetection(rightWristLocations):
    return rightWristLocations[0][1] - rightWristLocations[-1][1] > 100


# Configure the logger
logging.basicConfig(
    filename='debug_log.txt',  # Output file
    level=logging.DEBUG,       # Logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log message format
    filemode='w'  # Overwrite the log file each run (change to 'a' to append)
)

logger = logging.getLogger()


In [22]:

def mapTops(pose_points, img):
    # Calculate the distance between the shoulders (pose_points[0] and pose_points[1])
    shoulder_distance = math.sqrt((pose_points[1][0] - pose_points[0][0]) ** 2 + (pose_points[1][1] - pose_points[0][1]) ** 2)
    
    # Scale offsets based on shoulder distance
    shoulder_scale_factor = shoulder_distance / 250  # Adjust the denominator based on your needs; it's a scaling factor.
    shoulder_x_offset = int(100 * shoulder_scale_factor)
    hip_x_offset = int(180 * shoulder_scale_factor)
    shoulder_y_offset = int(50 * shoulder_scale_factor)
    
    # Calculate the height-to-width ratio of the input image
    heightToWidth = img.shape[0] / img.shape[1]
    width = int(math.fabs(pose_points[1][0] - pose_points[0][0]) * heightToWidth)
    height = int(width * heightToWidth)
    
    src = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype='float32')
    
    dest = np.array([
        [pose_points[0][0] + shoulder_x_offset, pose_points[0][1] - shoulder_y_offset], # Left shoulder
        [pose_points[1][0] - shoulder_x_offset, pose_points[1][1] - shoulder_y_offset], # Right shoulder
        [pose_points[3][0] - hip_x_offset, pose_points[3][1]], # Right hip
        [pose_points[2][0] + hip_x_offset, pose_points[2][1]] # Left hip
    ], dtype='float32')
    
    # Logging pose points and their mapping
    logger.debug(f"Og Pose Points: {pose_points}")
    logger.debug(f"Source Points (src): {src}")
    logger.debug(f"Destination Points (dest): {dest}")
    
    transform = cv2.getPerspectiveTransform(src, dest)
    img = cv2.resize(img, (width, height))
    
    return img, transform


def mapBottoms(pose_points, img):
    # Calculate the distance between the shoulders (pose_points[0] and pose_points[1])
    hip_distance = math.sqrt((pose_points[1][0] - pose_points[0][0]) ** 2 + (pose_points[1][1] - pose_points[0][1]) ** 2)
    
    # Scale offsets based on shoulder distance
    hip_scale_factor = hip_distance / 250  # Adjust the denominator based on your needs; it's a scaling factor.
    hip_x_offset = int(150 * hip_scale_factor) # 180 -> 150
    ankle_x_offset = int(150 * hip_scale_factor) # 200 -> 150
    # ankle_y_offset = int(200 * hip_scale_factor)
    hip_y_offset = int(70 * hip_scale_factor) # 10 -> 70
    
    # Calculate the height-to-width ratio of the input image
    heightToWidth = img.shape[0] / img.shape[1]
    width = int(math.fabs(pose_points[1][0] - pose_points[0][0]) * heightToWidth)
    height = int(width * heightToWidth)
    
    src = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype='float32')
    
    dest = np.array([
        [pose_points[0][0] + hip_x_offset, pose_points[0][1] - hip_y_offset], # Left shoulder
        [pose_points[1][0] - hip_x_offset, pose_points[1][1] - hip_y_offset], # Right shoulder
        [pose_points[3][0] - ankle_x_offset, pose_points[3][1]], # Right hip
        [pose_points[2][0] + ankle_x_offset, pose_points[2][1]] # Left hip
    ], dtype='float32')
    
    # Logging pose points and their mapping
    logger.debug(f"Og Pose Points: {pose_points}")
    logger.debug(f"Source Points (src): {src}")
    logger.debug(f"Destination Points (dest): {dest}")
    
    transform = cv2.getPerspectiveTransform(src, dest)
    img = cv2.resize(img, (width, height))
    
    return img, transform

def overlay_clothing(frame, clothing_img, transform):
    # Debugging: Shape of inputs before transformation
    logger.debug(f"Frame shape: {frame.shape}, Clothing image shape: {clothing_img.shape}")

    # Warp the clothing image to match the person's pose
    transformed_clothing = cv2.warpPerspective(clothing_img, transform, (frame.shape[1], frame.shape[0]))
    logger.debug(f"Transformed clothing image shape: {transformed_clothing.shape}")

    # Handle alpha blending if clothing image has an alpha channel
    if transformed_clothing.shape[-1] == 4:  # If the image has an alpha channel
        logger.debug("Clothing image has an alpha channel.")
        alpha_channel = transformed_clothing[:, :, 3] / 255.0  # Normalize alpha channel to 0-1
        for c in range(0, 3):  # Iterate over the RGB channels
            frame[:, :, c] = (1.0 - alpha_channel) * frame[:, :, c] + alpha_channel * transformed_clothing[:, :, c]
    else:
        logger.debug("Clothing image does not have an alpha channel.")
        # Directly overlay the transformed clothing (no transparency)
        mask = transformed_clothing > 0  # Consider any non-zero pixel as part of the clothing
        frame[mask] = transformed_clothing[mask]

    logger.debug("Overlay operation completed.")
    return frame

def capture_and_rotate_frame(camera, angle):
    ret, frame = camera.read()
    if not ret:
        return None

    # Rotate the frame according to the specified angle
    if angle == 90:
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif angle == -90:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif angle == 180:
        frame = cv2.rotate(frame, cv2.ROTATE_180)

    return frame

In [23]:
leftWristLocations = deque(maxlen=10)
rightWristLocations = deque(maxlen=10)

In [ ]:
desktop = False

cap = cv2.VideoCapture(0)

angle = 0 if desktop else -90


last_left_swipe_time = 0
last_right_swipe_time = 0
swipe_delay = 1

clothing_tops = ["White T-shirt", "Blue T-shirt", "Red T-shirt"]
clothing_bottoms = ["Green Pants", "Tan Slacks", "White Pants"]

clothing_tops_index = 0
clothing_bottoms_index = 0
clothing_top_selected = False


## Setup mediapipe instanceq
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    

    while True:
        # frame = capture_and_rotate_frame(cap, angle)
        
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame is None: 
            print("Frame capture returned None; ending loop")
            break
        
        #frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        listTops = os.listdir("top_images")
        listBottoms = os.listdir("bottom_images")
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
            
                # Get coordinates
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                norm_coord = [frame.shape[0], frame.shape[1]] if desktop else [frame.shape[1], frame.shape[0]]

                # Normalizes Coordinates
                left_shoulder_coords = tuple(np.multiply(left_shoulder, norm_coord).astype(int))
                right_shoulder_coords = tuple(np.multiply(right_shoulder, norm_coord).astype(int))

                left_hip_coords = tuple(np.multiply(left_hip, norm_coord).astype(int))
                right_hip_coords = tuple(np.multiply(right_hip, norm_coord).astype(int)) 
                
                left_ankle_coords = tuple(np.multiply(left_ankle, norm_coord).astype(int))
                right_ankle_coords = tuple(np.multiply(right_ankle, norm_coord).astype(int))

                upper_coords = [left_shoulder_coords, right_shoulder_coords, left_hip_coords, right_hip_coords]
                lower_coords = [left_hip_coords, right_hip_coords, left_ankle_coords, right_ankle_coords]

                #testing***
                
               
                for bone in bone_list:
                    if bone.name == "rightShoulder":
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
                        # left_elbow_coords = tuple(np.multiply(left_elbow, norm_coord).item())
                        y = left_elbow[1]-left_shoulder[1]
                        x = left_elbow[0]-left_shoulder[0]
                        z = left_elbow[2]-landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z
                        d = math.sqrt(math.pow(y,2)+math.pow(x,2))
                        #find rotation angle of the bone
                        
                        bone.rotation = [math.sin(math.asin(y/d)/2),0,math.sin(math.atan(x/z)/2),math.cos(math.asin(y/d)/2)]
                        # bone.translation = [0.078867807894,left_shoulder[0],left_shoulder[1]]
                        
                    elif bone.name == "leftShoulder":
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]
                        # right_elbow_coords = tuple(np.multiply(right_elbow, norm_coord).item())
                        y = right_elbow[1]-right_shoulder[1]
                        x = right_elbow[0]-right_shoulder[0]
                        z = right_elbow[2]-landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z
                        d = math.sqrt(math.pow(y,2)+math.pow(x,2))
                        #find rotation angle of the bone
                        
                        bone.rotation = [-math.sin(math.asin(y/d)/2),0,-math.sin(math.atan(x/z)/2),math.cos(math.asin(y/d)/2)]
                        # bone.translation = [-0.117773614, right_shoulder[0],right_shoulder[1]]
                
                    elif bone.name == "hip":#fix hip rotation
                        z = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z-landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z
                        y = right_hip[1]-left_hip[1]
                        x = right_hip[0]-left_hip[0]
                        d = math.sqrt(math.pow(y,2)+math.pow(x,2))
                        if(x>=0):
                            bone.rotation = [0,math.sin(math.atan(x/z)/2),0,math.cos(math.asin(y/d)/2)]
                        elif(x<0):
                            bone.rotation = [0,1+math.sin(math.atan(x/z)/2),0,math.cos(math.asin(y/d)/2)]
                        # bone.translation = [(left_hip[0] + right_hip[0])/2,(left_hip[1] + right_hip[1])/2, (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z + landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z)/2]
                
                    bone.translation = [float(val) for val in bone.translation]
                    bone.rotation = [float(val)/np.linalg.norm(bone.rotation) for val in bone.rotation]
                
                
                # 3D overlay
                scene.set_pose(node, pose=translation)

                # Render the scene
                color, _ = renderer.render(scene)

                # Display with OpenCV
                color_bgr = color[..., ::-1]  # Convert RGB to BGR
                cv2.imshow("GLTF Viewer", color_bgr)

                # Quit on pressing 'q'
                if cv2.waitKey(10) & 0xFF == ord("q"):
                    break

                # time.sleep(0.01)  # Slow down animation

                #end of testing***

                try:
                    gltf.save_binary("3D/magicmirror.glb")
                    # active_update()
                    
                except Exception as e:
                    print(f"Error saving GLB file: {e}")


                # 2D MAPPING LOGIC
                # for point in [left_shoulder_coords, right_shoulder_coords, left_hip_coords, right_hip_coords]:
                #     cv2.circle(frame, point, 5, (0, 255, 0), -1)

                # bottom_image = cv2.imread(os.path.join("bottom_images", listBottoms[clothing_bottoms_index]),cv2.IMREAD_UNCHANGED)
                # bottom_image, transform = mapBottoms(lower_coords, bottom_image)
                # frame = overlay_clothing(frame, bottom_image, transform)
                
                # top_image = cv2.imread(os.path.join("top_images", listTops[clothing_tops_index]),cv2.IMREAD_UNCHANGED)
                # top_image, transform = mapTops(upper_coords, top_image)
                # frame = overlay_clothing(frame, top_image, transform)

                # cv2.imshow("Overlayed Clothing", frame)
                # 2D MAPPING LOGIC END


                # 3D3D3D3D3D3D




                
                # 3D3D3D3D3D3D
                
               
                
                    
                
                # cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
               
            else:
                print("No pose landmarks detected")
                continue
            
            # Normalizes Coordinates
            left_wrist_coords = tuple(np.multiply(left_wrist, [640, 480]).astype(int))
            right_wrist_coords = tuple(np.multiply(right_wrist, [640, 480]).astype(int))

            leftWristLocations.append(left_wrist_coords)
            rightWristLocations.append(right_wrist_coords)
            
            current_time = time.time()

            if leftSwipeDetection(leftWristLocations):
                if current_time - last_left_swipe_time > swipe_delay:
                    last_left_swipe_time = current_time
                    if clothing_top_selected:
                        if clothing_tops_index == 0:
                            clothing_tops_index = len(clothing_tops) - 1
                        else:
                            clothing_tops_index -= 1
                    else:
                        if clothing_bottoms_index == 0:
                            clothing_bottoms_index = len(clothing_bottoms) - 1
                        else:
                            clothing_bottoms_index -= 1
                        
                
            if rightSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    last_right_swipe_time = current_time
                    if clothing_top_selected:
                        if clothing_tops_index == len(clothing_tops) - 1 :
                            clothing_tops_index = 0
                            
                        else:
                            clothing_tops_index += 1
                    else:
                        if clothing_bottoms_index == len(clothing_bottoms) - 1:
                            clothing_bottoms_index = 0
                        else:
                            clothing_bottoms_index += 1
                        
           
            if upSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    last_right_swipe_time = current_time
                    clothing_top_selected = not clothing_top_selected
                    if clothing_top_selected: #if the boolean is false that means bottom array selected else top array
                        print("top array")
                    else:
                        print("bottom array")

            # swipeDetection(leftWristLocations, rightWristLocations, last_left_swipe_time, last_right_swipe_time, last_up_swipe_time, current_time, clothing_tops_index, clothing_bottoms_index, clothing_top_selected)

        except Exception as e:
            print(f"An error occurred during pose estimation or clothing mapping: {e}")
        except:
            pass
            
       
        # Draw a rectangle at the top of the screen for clothing item display
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Display clothing item name
        cv2.putText(image, 'CLOTHING ITEM', (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        if(clothing_top_selected):
            cv2.putText(image, clothing_tops[clothing_tops_index], 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        else:
             cv2.putText(image, clothing_bottoms[clothing_tops_index], 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Render detections
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
        #                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
        #                          )
        
        # cv2.imshow('Rotated Frame', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
   

No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
N

c:\Users\mertz\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


top array
bottom array


KeyboardInterrupt: 